In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/YLP

/content/drive/MyDrive/YLP


In [ ]:
# Unless mentioned, code is from:
# Adversarial Example Generation, Inkawhich, N. and Uriegas, E.,  2018, Available from: https://github.com/pytorch/tutorials/blob/master/beginner_source/fgsm_tutorial.py , Accessed: 20 Feb 2022

In [4]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import datetime
import csv
from RKNet import RKNet

# NOTE: This is a hack to get around "User-agent" limitations when downloading MNIST datasets
#       see, https://github.com/pytorch/vision/issues/3497 for more information
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)


In [5]:
## JB Code Block ##

epsilons = [2**(-7), 2**(-6), 2**(-5), 2**(-4), 2**(-3), 2**(-2), 2**(-1)]
print(epsilons)


# filename for trained parabolic network
pretrained_model = os.path.join('results',"current_pdenet.pt")
use_cuda=True

In [7]:
## JB Code Block ##

# Transform to normalize 
shift = transforms.Normalize(
    mean=[0.5,0.5,0.5],
    std=[1.,1.,1.],
  )

# CIFAR-10 test dataset and dataloader declaration
test_data =  datasets.CIFAR10('../data', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            shift]))

test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

Files already downloaded and verified
CUDA Available:  True


In [9]:
## JB Code Block ##

# Load parabolic network
model = torch.load(pretrained_model)
# Load model to GPU
model = model.to(device)

In [10]:
# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    if epsilon == 0:
      return image
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [11]:
def test( model, device, test_loader, epsilon ):

    # Accuracy counter
    correct = 0
    adv_examples = []
    
    # JB code line: keep track of incorrect examples as they are not attacked
    incorrect_init = 0

  
    # Loop over all examples in test set
    for data, target in test_loader:

        data, target = data.to(device), target.to(device)


        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output = model(data)
        
        init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

        # If the initial prediction is wrong, dont bother attacking, just move on
        if init_pred.item() != target.item():
          incorrect_init += 1
          continue

        # Calculate the loss
        loss = F.nll_loss(output, target)

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Collect datagrad
        data_grad = data.grad.data

        # Call FGSM Attack
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        # Re-classify the perturbed image
        output = model(perturbed_data)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        if final_pred.item() == target.item():
            correct += 1
            # Special case for saving 0 epsilon examples
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        else:
            # Save some adv examples for visualization later
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )

    # Calculate final accuracy for this epsilon
    fgsm_tested = len(test_loader)-incorrect_init
    
    # JB code line: accuracy is only out of examples that were FGSM tested
    final_acc = correct/float(fgsm_tested)
    # Original line: final_acc= correct/float(len(test_loader))
    
    # JB code line: match print-out with changes in line above
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, fgsm_tested, final_acc))

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

In [12]:
accuracies = []
examples = []

# Run test for each epsilon
for eps in epsilons:
    acc, ex = test(model, device, test_loader, eps)
    accuracies.append(acc)
    examples.append(ex)

Epsilon: 0	Test Accuracy = 8471 / 8471 = 1.0
Epsilon: 0.0078125	Test Accuracy = 2738 / 8471 = 0.3232203990083815
Epsilon: 0.015625	Test Accuracy = 1820 / 8471 = 0.21485066698146618
Epsilon: 0.03125	Test Accuracy = 1356 / 8471 = 0.16007555188289457
Epsilon: 0.0625	Test Accuracy = 1199 / 8471 = 0.14154173061031755
Epsilon: 0.125	Test Accuracy = 1144 / 8471 = 0.13504899067406445
Epsilon: 0.25	Test Accuracy = 1022 / 8471 = 0.12064691299728486
Epsilon: 0.5	Test Accuracy = 861 / 8471 = 0.10164089245661669
